# Webscrapping with Selenium to fill the NAs 

## Looking for NAs and duplicates

In [65]:
import pandas as pd
import numpy as np
df = pd.read_csv('vgsales.csv', index_col = 0).copy()

In [66]:
df.isna().sum()

Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [67]:
df.duplicated().sum()

1

In [68]:
df[df.duplicated(keep=False) == True]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
15000,Wii de Asobu: Metroid Prime,Wii,NaN,Shooter,Nintendo,0.0,0.0,0.02,0.0,0.02
15002,Wii de Asobu: Metroid Prime,Wii,NaN,Shooter,Nintendo,0.0,0.0,0.02,0.0,0.02


In [69]:
df.drop_duplicates(inplace = True)
# I chose to keep the first.

In [70]:
df.duplicated().sum()

0

## Webscrapping

In [71]:
import warnings
warnings.filterwarnings('ignore')

In [72]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common import keys 
from time import sleep 
from tqdm import tqdm

In [73]:
year_na = df[df['Year'].isna()]

In [74]:
# for years
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.google.com')
driver.find_element(By.ID, 'L2AGLb').click()
for i, row in tqdm(year_na.iterrows(), total=len(year_na)):
    bar = driver.find_element(By.NAME, 'q')
    bar.clear()
    sleep(2)
    bar.send_keys(row['Name'], ' ', row['Platform'], ' year')
    sleep(1)
    driver.find_element(By.NAME, 'q').send_keys(keys.Keys.ENTER)
    try:
        year_box = driver.find_element(By.CLASS_NAME, 'kp-hc').text
        year_na.loc[i, 'Year'] = year_box[-4:]
    except: 
        year_na.loc[i, 'Year'] = np.nan


100%|██████████| 270/270 [22:25<00:00,  4.98s/it]


In [75]:
df['Year'].fillna(year_na['Year'],inplace = True)

In [76]:
df.to_csv('vgsales_webscrap.csv')

In [78]:
df.isna().sum()

Name             0
Platform         0
Year            86
Genre            0
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
Global_Sales     0
dtype: int64